In [ ]:
PATH_DATASET = 'XXX'
PATH_RELACOES_JURIDICAS = 'XXX'

API_KEY = 'XXX'
MODELO_GPT = 'gpt-4o-mini'

##Dados 

In [2]:
import pandas as pd

df_assuntos = pd.read_csv('/Users/raquelsilveira/Downloads/predicoes_dataset_teste_bert_ap.csv', nrows=1)
dict_assuntos = eval(df_assuntos.iloc[0].label_dict)
assuntos = [dict_assuntos[k] for k in dict_assuntos]
len(assuntos)

310

In [ ]:
import pandas as pd

df_data = pd.read_csv(PATH_DATASET)
df_teste = df_data[df_data.subset == 'test']

## Inicialização do DSPy

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = API_KEY

In [ ]:
import dspy

model = MODELO_GPT
max_tokens = 4096
gpt = dspy.OpenAI(model=model,
                max_tokens=max_tokens,
                  model_type='chat'
                  )
dspy.settings.configure(lm=gpt, trace=[], temperature=0.0)

## Classificação de Assuntos

In [ ]:
import pandas as pd

df_relacoes_juridicas = pd.read_csv(PATH_RELACOES_JURIDICAS)
df_relacoes_juridicas = df_relacoes_juridicas[pd.notna(df_relacoes_juridicas['Relação'])]
df_relacoes_juridicas

In [ ]:
import dsp
import dspy
from dspy.signatures.signature import ensure_signature

from dspy.predict import Predict

class ChainOfThought(Predict):
    def __init__(self, signature, desc='', rationale_type=None, activated=True, **config):
        super().__init__(signature, **config)

        self.activated = activated

        signature = ensure_signature(self.signature)
        *_keys, last_key = signature.output_fields.keys()

        rationale_type = rationale_type or dspy.OutputField(

            prefix=f"Raciocínio: Vamos pensar passo-a-passo em ordem para",
            desc=f"{desc} Nesta petição inicial ...",
        )

        self.extended_signature = signature.prepend("rationale", rationale_type, type_=str)

    def forward(self, **kwargs):
        new_signature = kwargs.pop("new_signature", None)
        if new_signature is None:
            if self.activated is True or (
                self.activated is None and isinstance(dsp.settings.lm, dsp.GPT3)
            ):
                signature = self.extended_signature
            else:
                signature = self.signature
        else:
            signature = new_signature
            # template = dsp.Template(self.signature.instructions, **new_signature)
        return super().forward(signature=signature, **kwargs)


    def dump_state(self):
        state = super().dump_state()

        # Cache the signature instructions and the last field's name.
        state["extended_signature_instructions"] = self.extended_signature.instructions

        *_, last_key = self.signature.fields.keys()
        state["extended_signature_prefix"] = self.extended_signature.fields[last_key].json_schema_extra['prefix']

        return state

    def load_state(self, state):
        super().load_state(state)

        # Reconstruct the signature.
        if "extended_signature_instructions" in state:
            instructions = state["extended_signature_instructions"]
            self.extended_signature = self.extended_signature.with_instructions(instructions)

        if "extended_signature_prefix" in state:
            prefix = state["extended_signature_prefix"]
            *_, last_key = self.extended_signature.fields.keys()
            self.extended_signature = self.extended_signature.with_updated_fields(last_key, prefix=prefix)

In [ ]:
n1 = []
for item in list(df_data.label.unique()):
  aux_n1 = item.split('/')[0].split('-')[1].strip()
  if aux_n1 not in n1:
    n1.append(aux_n1)

print('n1:', n1)

In [ ]:
class ResumoAssinatura(dspy.Signature):
        '''
        *** INFORMAÇÕES IMPORTANTES ***
        Aqui estão algumas das informações mais relevantes de um processo judicial:

        1. **Qualificação das Partes**:
        - **Natureza jurídica básica das pessoas**: É pessoa física ou jurídica? É pessoa pública ou privada?
        - **Estado civil**: Pode ser relevante em casos de direito de família, sucessões, etc.
        - **Profissão e ocupação**: Pode ser relevante em questões trabalhistas, civis e até criminais.
        - **Atividade Econômica (no caso de pessoas jurídicas)**: Ajuda na identificação do tipo de serviço que pode está sendo prestado em casos de direito do consumidor.

        2. **Natureza da Relação Jurídica**:
        - **Qualidade recíproca das partes**: O que os envolvidos são uns para os outros dentro da relação jurídica? São consumidores? São fornecedores? Se fornecedores, fornecem quais tipos de produtos e serviços? São contratantes? São proprietários de imóvel? São servidores do estado? São pessoas asseguradas pela previdência social? 
        - **Objeto do litígio**: Em torno de quê gira a relação entre as partes (e.g., dívida, posse de bem, a parentalidade, a guarda de filhos, um contrato, um plano de saúde, um seguro de carro, um benefício previdenciário, um contrato de trabalho etc.).

        3. **Fatos Alegados pelas Partes**:
        - **Descrição do evento ou comportamento que motivou o processo**: Esclarece o que teria gerado o conflito entre as partes, sendo essencial para entender a origem da ação. Por exemplo, em uma ação de indenização por acidente de trânsito, os fatos envolvem a dinâmica do acidente, os danos causados, e o comportamento das partes no momento do evento.
        - **Causa de pedir próxima e remota**: A causa próxima refere-se ao fato específico que levou à ação (exemplo: inadimplência em um contrato), enquanto a causa remota pode ser a relação anterior entre as partes (exemplo: a assinatura de um contrato de prestação de serviços) ou qualquer outro fundamento jurídico para a pretensão (exemplo: a existência de lei que dá ao consumidor o direito de ter devolvido em dobro o valor pago indevidamente a um fornecedor)

        4. **Base Legal:
        - **Dispositivos legais aplicáveis**: Determina os artigos de lei ou regulamentos que a parte autora cita como fundamento para a demanda. Por exemplo, em uma ação trabalhista, o autor pode basear a demanda em direitos garantidos pela CLT (Consolidação das Leis do Trabalho), como o pagamento de horas extras.

        5. **Provas Apresentadas**:
        - **Documentos, depoimentos, perícias**: As provas anexadas ao processo são fundamentais para entender o porquê da ação e como as partes tentam sustentar suas alegações. Por exemplo, laudos médicos, testemunhas oculares, documentos contratuais, e-mails trocados, entre outros.

        6. **Pedido**:
        - **O que o autor pretende obter com o processo**: É o pedido formal que o autor faz a um juiz, como uma condenação ao pagamento de uma quantia, a entrega de um bem, ou uma obrigação de fazer ou não fazer.
                Exemplo: Em uma ação de despejo, o pedido é a desocupação do imóvel por falta de pagamento de aluguel.

        7. **Contestação da Parte Ré**:
        - **Argumentos de defesa e fundamentos legais**: A defesa apresenta seus motivos para justificar a improcedência da ação, podendo contestar os fatos, a aplicação da lei ou as provas apresentadas. Por exemplo, em uma ação de indenização por danos morais, a defesa pode argumentar que não houve ato ilícito ou nexo causal entre o suposto ato e o dano alegado.

        8. **Outros Aspectos Relevantes**:
        - **Grupos vulneráveis**: Se uma das partes pertence a um grupo vulnerável, como idosos, menores, pessoas com deficiência, etc.
        - **Precedentes vinculantes**: De que maneira decisões anteriores (sobretudo dos Tribunais Superiores) vinculam ou influenciam aquele caso específico.

        *** INSTRUÇÕES PASSO-A-PASSO ***
        Com base nas ```INFORMAÇÕES IMPORTANTES``` apresentadas anteriormente, use as instruções passo-a-passo a seguir para gerar uma descrição do processo.

        Passo 1 - Extraia do processo as informações importantes sobre as partes do processo.

        Passo 2 - Leia novamente o processo e acrescente ao resultado do passo anterior detalhes que ainda não estavam presentes. Mantenha o resultado desse passo em no MÍNIMO 500 e no MÁXIMO 700 palavras.

        Passo 3 - Leia mais uma vez o processo e acrescente ao resultado do ```Passo 2`` detalhes que ainda não estavam presentes. Mantenha o resultado desse passo em no MÍNIMO 400 e no MÁXIMO 500 palavras.

        Passo 4 - Mostre como saída APENAS uma descrição do processo gerada a partir do resultado obtido no ```Passo 3``` em um ÚNICO parágrafo.

        Leia atentamente o ```TEXTO DO PROCESSO``` que será apresentado a seguir e gere uma descrição do processo através das ```INSTRUÇÕES PASSO-A-PASSO``` descritas anteriormente.
        '''
        
        texto = dspy.InputField(desc="```TEXTO DO PROCESSO```")
        descricao_processo = dspy.OutputField(desc="Descrição do processo")

In [17]:
class ResumoModulo(dspy.Module):
    def __init__(self):
        super().__init__()
        desc_cot = 'gerar uma descrição do processo.'
        self.gera_resumo = ChainOfThought(ResumoAssinatura, desc=desc_cot)

    def forward(self, texto):
        pred = self.gera_resumo(texto=texto)
        return pred

In [ ]:
class RelacaoJuridicaAssinatura(dspy.Signature):
    # '''
    # Com base no texto apresentado, classifique a relação jurídica entre as partes do caso em questão.
    # Extraia a relação jurídica entre as partes do caso e classifique em uma das seguintes categorias. Cada categoria está descrita na entrada dos dados.
    # Leia atentamente a ```DESCRIÇÃO DO CASO``` que será apresentado a seguir e classifique a relação jurídica entre as partes presentes no caso.
    # '''
    
    '''
    *** INSTRUÇÕES PASSO-A-PASSO ***
    Com base no texto apresentado, classifique a relação jurídica entre as partes do caso em questão.

    Passo 1: Extraia a relação jurídica entre as partes do caso e classifique em uma das categorias informadas na entrada de dados.

    Passo 2: Pense passo-a-passo e formate a sua resposta apresentando o seu raciocínio em no MÍNIMO 200 PALAVRAS e a categoria obtida no ```Passo 1```. 

    Leia atentamente a ```DESCRIÇÃO DO PROCESSO``` que será apresentado a seguir e utilize as ```INSTRUÇÕES PASSO-A-PASSO``` para classificar a relação jurídica entre as partes presentes no caso.
    '''
    
    descricao_processo = dspy.InputField(desc="```DESCRIÇÃO DO PROCESSO```")
    categorias = dspy.InputField(
        desc="Categorias para escolher a relação jurídica do caso em questão.")
    descricao_categorias = dspy.InputField(
        desc="Descrição das categorias das relações jurídicas.")
    relacao_juridica = dspy.OutputField(desc="Relação jurídica entre as partes do caso em questão. Informe apenas a relação jurídica")

In [ ]:
import difflib

def valida_relacao_juridica(pred, opcoes):
  
  pred.relacao_juridica = pred.relacao_juridica.replace('Relação Juridica: ', '')

  #Verifica se a predição é igual sintaticamente em 90%
  if len(difflib.get_close_matches(pred.relacao_juridica, opcoes, n=1, cutoff=0.9)) > 0:
    pred.relacao_juridica = difflib.get_close_matches(pred.relacao_juridica, opcoes, n=1, cutoff=0.9)[0]
    return True

  return False

class RelacaoJuridicaModulo(dspy.Module):
    def __init__(self):
        super().__init__()
        desc_cot = 'identificar a relação jurídica entre as partes do caso em questão.'
        self.classifica_relacao_juridica = ChainOfThought(RelacaoJuridicaAssinatura, desc=desc_cot)

    def forward(self, texto, opcoes, descricoes):
        opcoes_relacoes_juridicas = ' ou '.join(opcoes)
        opcoes_desc_relacoes_juridicas = ';\n'.join([f'{opcao}: {descricao}' for opcao, descricao in zip(opcoes, descricoes)])
        pred = self.classifica_relacao_juridica(descricao_processo=texto, categorias=opcoes_relacoes_juridicas, descricao_categorias=opcoes_desc_relacoes_juridicas)

        dspy.Suggest(result=valida_relacao_juridica(pred, opcoes),
                     msg=f'A relação jurídica deve ser uma das seguintes categorias: {str(opcoes)}')

        return pred

In [ ]:
class PedidoAssinatura(dspy.Signature):
    # '''
    # Leia atentamente o RESUMO DO CASO que será apresentado a seguir e identifique todos os pedidos que se encaixam neste documento. 
    # Leia atentamente as definições dos pedidos presentes na entrada de dados e use-as para retornar todos os pedidos que representem o caso. 
    # Apresente no mínimo um pedido e certifique-se que todos os pedidos que forem apresentados representem exatamente o caso. 
    # '''
    
    '''
    *** INSTRUÇÕES PASSO A PASSO ***  
    Com base na ```DESCRIÇÃO DO PROCESSO``` apresentada, identifique e extraia os pedidos mencionados no texto seguindo os passos abaixo:  

    Passo 1: Analise cuidadosamente as definições fornecidas a seguir e identifique os pedidos presentes no caso.  

    Passo 2: Garanta que a lista gerada no ```Passo 1``` contenha pelo menos um pedido.  

    Passo 3: Revise novamente a ```DESCRIÇÃO DO PROCESSO``` para verificar se algum pedido foi omitido na primeira análise. Se encontrar pedidos adicionais, inclua-os na lista.  

    Agora, leia atentamente a ```DESCRIÇÃO DO PROCESSO``` apresentada a seguir e siga as ```INSTRUÇÕES PASSO A PASSO``` para extrair e listar corretamente os pedidos mencionados no texto.
    '''

    descricao_processo = dspy.InputField(desc='```DESCRIÇÃO DO PROCESSO```')
    pedido = dspy.OutputField(desc='Pedido(s) do processo em questão')

In [ ]:
import difflib
import re

class PedidoModulo(dspy.Module):
    def __init__(self):
        super().__init__()
        desc_cot = 'identificar todos os pedidos do processo em questão.'
        self.extrai_pedidos = ChainOfThought(PedidoAssinatura, desc=desc_cot)

    def forward(self, texto):
        pred = self.extrai_pedidos(descricao_processo=texto)
        
        return pred

In [ ]:
class ClassificaAssuntoPrincipalAssinatura(dspy.Signature):
   
   # """
   # Com base no texto apresentado, classifique o assunto principal do caso em questão.

   # Extraia o assunto do caso e classifique em um dos assuntos informados. Cada assunto está apresentado na entrada dos dados.

   # Leia atentamente a ```PEDIDOS``` e o ```MOTIVO``` do caso em questão que serão apresentados a seguir e classifique o assunto principal do caso.
   # """
   
   '''
   *** INSTRUÇÕES PASSO-A-PASSO *** 
   Com base nos ```PEDIDOS``` que levaram à instauração do processo, 
   siga as instruções abaixo para identificar o **assunto principal** do processo jurídico em questão, 
   dentro da lista fornecida de opções de assuntos. 
   
   ### **Passo 1 - Análise do Processo** 
   - Leia atentamente os ```PEDIDOS``` que levaram à instauração do processo. 
   - Se houver múltiplos pedidos, determine qual é o pedido mais relevante que motiva o processo. 
   - Priorize a essência do direito ou benefício envolvido no pedido, e não apenas aspectos processuais como descontos indevidos ou supressões. 
   
   ### **Passo 2 - Seleção do Assunto Principal** 
   - Compare os ```PEDIDOS``` com os **assuntos disponíveis na lista fornecida**. 
   - Escolha o **assunto mais específico e relevante**, garantindo que ele representa o assunto principal do processo. 
   - Se houver um pedido principal e pedidos acessórios, priorize o pedido principal. 
   
   ### **Passo 3 - Garantia de Consistência** 
   - O assunto selecionado **deve estar presente na lista de assuntos fornecida**. 
   - Compare **toda a estrutura hierárquica** do assunto antes de escolher a melhor opção 
   - Caso o processo envolva múltiplos temas, escolha **aquele que melhor representa o assunto principal** e não os temas secundários. 
   
   ### **Passo 4 - Retorno da Resposta** 
   - Retorne apenas o código e a descrição do assunto principal conforme a estrutura da lista. 
   - **Exemplo de saída correta**: 
   - 8826 - DIREITO PROCESSUAL CIVIL E DO TRABALHO/11781 - Objetos de cartas precatórias cíveis/de ordem/11783 - Citação 
   - Além disso, é importante que você considere a relevância e a especificidade do assunto escolhido, evitando generalizações. 
   
   Leia os ```PEDIDOS``` e identifique o **assunto principal** do processo conforme as instruções acima, sempre buscando a opção que melhor se alinha com a essência do caso apresentado.
   '''
   
   pedidos = dspy.InputField(desc="```PEDIDOS``` do processo em questão")
   opcoes_assunto = dspy.InputField(desc='''Opções para escolha do assunto principal do processo.''')

   assunto_principal = dspy.OutputField(desc=f"A saída deve ser o assunto principal do processo, que é um dos seguintes assuntos {opcoes_assunto}")

In [ ]:
opcoes_assuntos = {}
def extrai_assunto_ramo_direito(n1):
  for n in n1:
    ramo_direito = n.split('(')[0].strip()
    opcoes_ramo_direito = []
    for assunto in assuntos:
      if " - " + ramo_direito + '/' in assunto:
        opcoes_ramo_direito.append(assunto)
    opcoes_assuntos[ramo_direito] = opcoes_ramo_direito
  return opcoes_assuntos

opcoes_assuntos = extrai_assunto_ramo_direito(n1)
print(opcoes_assuntos)

def valida_assunto_principal(pred, opcoes):
    if len(difflib.get_close_matches(pred.assunto_principal, opcoes, n=1, cutoff=0.9)) > 0:
      pred.assunto = difflib.get_close_matches(pred.assunto_principal, opcoes, n=1, cutoff=0.9)[0]
      return True

    for o in opcoes:
      if o.endswith(f'/{pred.assunto_principal}'):
        pred.assunto = o
        return True

    return False

class ClassificadorAssuntoPrincipalModulo(dspy.Module):
    def __init__(self):
        super().__init__()
        desc_cot = 'identificar o **assunto principal** do processo conforme as instruções, sempre buscando a opção que melhor se alinha com a essência do caso apresentado'
        self.classifica_assunto = ChainOfThought(ClassificaAssuntoPrincipalAssinatura, desc=desc_cot)

    def forward(self, pedido, opcoes):
        pred = self.classifica_assunto(pedidos=pedido,
                                       opcoes_assunto=str(opcoes))
        
        pred.assunto_principal = pred.assunto_principal.replace('Assunto: ', '')
        if len(opcoes) == 1:
          dspy.Suggest(valida_assunto_principal(pred, opcoes), f'O assunto principal da petição inicial deve ser o seguinte assunto: {opcoes}. Apenas um dos assuntos dado nas opções de assunto. Sem nenhum caractere adicional!')
        else:
          dspy.Suggest(valida_assunto_principal(pred, opcoes), f'O assunto principal da petição inicial deve ser um dos seguintes assuntos: {opcoes}. Cada assunto está separado por vírgula. Apenas um dos assuntos dado nas opções de assunto. Sem nenhum caractere adicional!')

        return pred

{'DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DIREITO PÚBLICO': ['9985 - DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DIREITO PÚBLICO/10219 - Servidor Público Civil/10288 - Sistema Remuneratório e Benefícios/10706 - Gratificações Municipais Específicas', '9985 - DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DIREITO PÚBLICO/9997 - Atos Administrativos/10022 - Infração Administrativa/10023 - Multas e demais Sanções', '9985 - DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DIREITO PÚBLICO/9997 - Atos Administrativos/10011 - Improbidade Administrativa/10012 - Dano ao Erário', '9985 - DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DIREITO PÚBLICO/10219 - Servidor Público Civil/10288 - Sistema Remuneratório e Benefícios/10296 - Descontos Indevidos', '9985 - DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DIREITO PÚBLICO/10219 - Servidor Público Civil/10288 - Sistema Remuneratório e Benefícios/10312 - Piso Salarial', '9985 - DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DIREITO PÚBLICO/10219 - Servidor Públi

In [ ]:
import pandas as pd
import dspy

mapa_relacao_ramo = {
    'Relação cível': 'DIREITO CIVIL',
    'Relação processual': 'DIREITO PROCESSUAL CIVIL E DO TRABALHO',
    'Relação de consumo': 'DIREITO DO CONSUMIDOR',
    'Relação penal': 'DIREITO PENAL',
    'Relação tributária': 'DIREITO TRIBUTÁRIO',
    'Relação de direito à saúde': 'DIREITO DA SAÚDE',
    'Relação administrativa': 'DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DIREITO PÚBLICO',
    'Relação trabalhista': 'DIREITO PROCESSUAL CIVIL E DO TRABALHO',
}

class ClassificadorAssunto(dspy.Module):

    def __init__(self):
        super().__init__()
        self.classifica_relacao_juridica = RelacaoJuridicaModulo().activate_assertions(max_backtracks=2)
        self.gera_resumo = ResumoModulo()
        self.extrai_pedido = PedidoModulo()
        self.classifica_assunto = ClassificadorAssuntoPrincipalModulo().activate_assertions(max_backtracks=2)
        

    def obter_ramo_direito_relacao_juridica(self, relacoes_juridicas):
        for m in mapa_relacao_ramo:
            if m in relacoes_juridicas:
                return mapa_relacao_ramo[m]
        return ''

    def obter_relacoes_hierarquicas(self, df_relacoes_juridicas, texto):
        relacao_ramo = []
        relacoes_juridicas = df_relacoes_juridicas[pd.isna(df_relacoes_juridicas['Relação Pai'])]
        opcoes_relacoes_juridicas = relacoes_juridicas['Relação'].tolist()
        descricoes_relacoes_juridicas = relacoes_juridicas['Verdade de base'].tolist()

        while not relacoes_juridicas.empty:
            nova_relacao_juridica = self.classifica_relacao_juridica(
                texto=texto,
                opcoes=opcoes_relacoes_juridicas,
                descricoes=descricoes_relacoes_juridicas
            ).relacao_juridica

            if nova_relacao_juridica not in opcoes_relacoes_juridicas or (relacao_ramo and nova_relacao_juridica == relacao_ramo[-1]):
                break

            relacao_ramo.append(nova_relacao_juridica)
            relacoes_juridicas = df_relacoes_juridicas[df_relacoes_juridicas['Relação Pai'] == nova_relacao_juridica]
            opcoes_relacoes_juridicas = relacoes_juridicas['Relação'].tolist()
            descricoes_relacoes_juridicas = relacoes_juridicas['Verdade de base'].tolist()

        return relacao_ramo

    def forward(self, texto: str):
        #Resumo
        resumo = self.gera_resumo(texto).descricao_processo
        
        if resumo == 'Desculpe, mas não posso ajudar com isso.':
            return None
        
        #Relação Jurídica
        relacao_juridica = self.obter_relacoes_hierarquicas(df_relacoes_juridicas, resumo)
        ramo_direito = self.obter_ramo_direito_relacao_juridica(relacao_juridica)
        opcoes = opcoes_assuntos.get(ramo_direito, [])
        
        #Pedido
        pred_pedido = self.extrai_pedido(texto=resumo)
        pedido = pred_pedido.pedido
        
        
        pred = self.classifica_assunto(pedido=pedido, opcoes=opcoes)
        return dspy.Prediction(
            descricao = resumo,
            relacoes_juridicas=relacao_juridica,
            ramo_direito=ramo_direito,
            pedido=pedido,
            assunto=pred.assunto_principal,
            rationale=pred.rationale if hasattr(pred, 'rationale') else ''
        )

In [ ]:
import tqdm

resultado = []

classifica_assunto = ClassificadorAssunto()

for index, row in tqdm.tqdm(df_teste.iterrows()):
  try:
    pred = classifica_assunto(texto=row.texto_peticao_inicial)
    
    resultado.append({'codprocesso': row.codprocesso,
                      'assunto_predito': pred.assunto,
                      'label': row.label,
                      'descricao_processo': pred.descricao,
                      'relacoes_juridicas': pred.relacoes_juridicas,
                      'ramo_direito': pred.ramo_direito,
                      'pedido': pred.pedido,
                      })

  except:
    print('erro', row.codprocesso)

df_resultado_gpt = pd.DataFrame(resultado)
df_resultado_gpt

In [ ]:
from sklearn import metrics

df_resultado_gpt['n1_usuario'] = df_resultado_gpt.label.apply(lambda x: x.split('/')[0])
n1 = df_resultado_gpt.n1_usuario.unique()

dados = []
for n in n1:
    df_filtrado = df_resultado_gpt[df_resultado_gpt.n1_usuario == n]
    dados.append({'Ramo do direito': n,
                  'f1-score': metrics.f1_score(df_filtrado.label, df_filtrado.assunto_predito, average='micro')})

print(pd.DataFrame(dados))
print(print(metrics.classification_report(df_resultado_gpt.label, df_resultado_gpt.assunto_predito)))